# Lezione 2: Agenti Avanzati e Integrazioni

Benvenuti alla seconda lezione! In questo notebook imparerete:

1. 🌐 **Ricerca Web**: Agente con Tavily per cercare informazioni online
2. 📝 **Summarize Middleware**: Riassumere conversazioni lunghe automaticamente
3. 🤝 **Human-in-the-Loop**: Intervento umano nelle decisioni dell'agente
4. 🗄️ **SQL Agent**: Interrogare database con linguaggio naturale
5. 📚 **RAG**: Retrieval Augmented Generation con PDF e Faiss

## Setup Iniziale

Verifichiamo l'ambiente e carichiamo le dipendenze.

In [1]:
import sys
import os
from dotenv import load_dotenv

# Carica variabili d'ambiente
load_dotenv()

print(f"Python version: {sys.version}")

# Verifica API keys
if os.getenv("OPENAI_API_KEY"):
    print("✅ OpenAI API key trovata")
if os.getenv("TAVILY_API_KEY"):
    print("✅ Tavily API key trovata")
else:
    print("⚠️ Tavily API key non trovata - necessaria per ricerca web")
    print("   Registrati su https://tavily.com per ottenere una chiave gratuita")

Python version: 3.13.11 (main, Dec 17 2025, 21:09:15) [MSC v.1944 64 bit (AMD64)]
✅ OpenAI API key trovata
✅ Tavily API key trovata


In [2]:
# Inizializza il modello
from langchain.chat_models import init_chat_model

model = init_chat_model(
    "openai:gpt-4o-mini",  # Usiamo un modello più potente per questi task
    temperature=0
)

print("✅ Modello inizializzato")

✅ Modello inizializzato


In [3]:
# con Cerebras
from langchain_cerebras import ChatCerebras

model = ChatCerebras(model_name="gpt-oss-120b", temperature=0)

## 1. 🌐 Agente con Ricerca Web (Tavily)

**Tavily** è un motore di ricerca ottimizzato per LLM che fornisce risultati strutturati e pertinenti.

**Vantaggi:**
- Risultati ottimizzati per AI (non HTML grezzo)
- Più veloce di Google Search
- Filtra automaticamente contenuti irrilevanti
- Piano gratuito disponibile

In [4]:
# Installazione Tavily (se necessario)
# uv pip install langchain-community tavily-python

from langchain_community.tools.tavily_search import TavilySearchResults

# Creiamo il tool di ricerca
search = TavilySearchResults(
    max_results=3,  # Numero massimo di risultati
    search_depth="advanced",  # "basic" o "advanced"
    include_answer=True,  # Include una risposta sintetica
    include_raw_content=False,  # Non includere HTML grezzo
)

print("✅ Tool Tavily Search creato")
print(f"Nome tool: {search.name}")
print(f"Descrizione: {search.description}")

✅ Tool Tavily Search creato
Nome tool: tavily_search_results_json
Descrizione: A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.


C:\Users\pigno\AppData\Local\Temp\ipykernel_15452\408747139.py:7: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search = TavilySearchResults(


In [5]:
# Test diretto del tool
result = search.invoke({"query": "Ultime notizie sull'intelligenza artificiale 2026"})

print("📰 Risultati della ricerca:\n")
for i, res in enumerate(result, 1):
    print(f"{i}. {res.get('title', 'N/A')}")
    print(f"   URL: {res.get('url', 'N/A')}")
    print(f"   Snippet: {res.get('content', 'N/A')[:150]}...\n")

📰 Risultati della ricerca:

1. Ces 2026, l'intelligenza artificiale è dappertutto. Ma bisogna saperla ...
   URL: https://www.wired.it/article/ces-2026-intelligenza-artificiale-banco-di-prova-novita/
   Snippet: È plausibile che molti nel settore siano in attesa di vedere che forma prenderà la strategia di OpenAI sui dispositivi. L’azienda ha già fatto sapere ...

2. 2026, quando l'Intelligenza artificiale diventa infrastruttura mentale e ...
   URL: https://www.rainews.it/articoli/2025/12/2026-quando-lintelligenza-artificiale-diventa-infrastruttura-mentale-e-ridefinisce-il-potere-umano-ecdeaa4d-e460-42a4-96fc-b87621606277.html
   Snippet: possibile. Questo atteggiamento porta a un'omologazione del pensiero e a una perdita di identità creativa. La discriminante sarà quindi la capacità di...

3. CES 2026: rivoluzione o marketing? L'intelligenza artificiale alla ...
   URL: https://www.ilsole24ore.com/art/ces-2026-rivoluzione-o-marketing-l-intelligenza-artificiale-prova-fatti-AI6JHjc
   

In [6]:
# Creiamo un agente con ricerca web usando la nuova API
from langchain.agents import create_agent

search_agent = create_agent(
    model=model,
    tools=[search],
    system_prompt="""Sei un assistente di ricerca intelligente.
    
Quando l'utente fa una domanda:
1. Usa il tool di ricerca per trovare informazioni aggiornate
2. Analizza i risultati e sintetizzali
3. Fornisci una risposta completa citando le fonti
4. Se le informazioni non sono sufficienti, dillo chiaramente

Rispondi sempre in italiano."""
)

print("✅ Agente di ricerca web creato con la nuova API!")

✅ Agente di ricerca web creato con la nuova API!


In [7]:
# Testiamo l'agente di ricerca
response = search_agent.invoke({
    "messages": "Quali sono le novità più importanti di LangChain nel 2026?"
})

print("🤖 Risposta dell'agente:\n")
print(response["messages"][-1].content)

🤖 Risposta dell'agente:

**Novità più importanti di LangChain nel 2026**

Nel 2026 il framework **LangChain** ha consolidato la sua posizione di riferimento per la costruzione di agenti intelligenti, passando da una fase di rapida sperimentazione a una maturità orientata all’uso enterprise. Le principali innovazioni introdotte (o in fase di consolidamento) sono le seguenti:

| Area | Novità / Evoluzione | Impatto pratico |
|------|----------------------|-----------------|
| **LangGraph** (sottoprogetto di LangChain) | • È passato dalla fase “beta” (maggio 2025) a una piattaforma stabile e più **maturata**. <br>• Supporta workflow a lungo termine, checkpoint persistenti e **pausa per approvazione umana**. <br>• Introduzione di protocolli **A2A (Agent‑to‑Agent)** e **MCP** per la comunicazione cross‑framework. | Consente di costruire agenti complessi che possono continuare l’esecuzione anche dopo riavvii, gestire flussi di lavoro regolamentati e collaborare con altri agenti in modo stand

## 2. 📝 Summarize Middleware

Per conversazioni lunghe, la **SummarizationMiddleware** riassume automaticamente i messaggi più vecchi per risparmiare token e mantenere il contesto gestibile.

**Quando usarlo:**
- Conversazioni molto lunghe (> 20-30 messaggi)
- Limiti di contesto del modello
- Costi elevati per token

- Token counter configurabile

**Novità con la nuova API:**- Gestione automatica di coppie AI/Tool message

- Middleware ufficiale da `langchain.agents.middleware.summarization`- Supporto per trigger multipli (token, messaggi, frazione)

In [8]:
# Esempio di configurazione avanzata con trigger multipli
from langchain.agents.middleware.summarization import SummarizationMiddleware

# Trigger quando SI VERIFICA UNA delle condizioni:
advanced_summarization = SummarizationMiddleware(
    model=model,
    trigger=[
        ("messages", 50),   # O quando raggiungi 50 messaggi
        ("tokens", 4000),   # O quando raggiungi 4000 token
        #("fraction", 0.8)   # O quando usi l'80% del contesto del modello se supportato
    ],
    keep=("tokens", 2000),  # Mantieni gli ultimi 2000 token
    # token_counter: funzione custom per contare i token (opzionale)
)

print("✅ SummarizationMiddleware avanzato creato")
print("\n🔮 Modalità di trigger disponibili:")
print("   - ('messages', N): Numero di messaggi")
print("   - ('tokens', N): Numero di token assoluti")
print("   - ('fraction', F): Frazione del contesto del modello (0.0-1.0)")
print("\n🔮 Modalità di keep disponibili:")
print("   - ('messages', N): Mantieni N messaggi recenti")
print("   - ('tokens', N): Mantieni N token recenti")
print("   - ('fraction', F): Mantieni F frazione del contesto")

✅ SummarizationMiddleware avanzato creato

🔮 Modalità di trigger disponibili:
   - ('messages', N): Numero di messaggi
   - ('tokens', N): Numero di token assoluti
   - ('fraction', F): Frazione del contesto del modello (0.0-1.0)

🔮 Modalità di keep disponibili:
   - ('messages', N): Mantieni N messaggi recenti
   - ('tokens', N): Mantieni N token recenti
   - ('fraction', F): Mantieni F frazione del contesto


In [9]:
from langgraph.checkpoint.memory import MemorySaver
from langchain.agents import create_agent
from langchain.agents.middleware.summarization import SummarizationMiddleware

# Creiamo un checkpointer per la memoria
memory = MemorySaver()

# Creiamo il middleware di summarization
summarization_middleware = SummarizationMiddleware(
    model=model,
    # Trigger: quando la conversazione raggiunge 10 messaggi
    trigger=("messages", 5),
    # Keep: mantieni gli ultimi 2 messaggi dopo il riassunto
    keep=("messages", 2)
)


print("✅ Middleware di summarization creato")
print(f"   - Keep: {summarization_middleware.keep}")
print(f"   - Trigger: {summarization_middleware.trigger}")

✅ Middleware di summarization creato
   - Keep: ('messages', 2)
   - Trigger: ('messages', 5)


In [10]:
# Creiamo un agente con summarization middleware
agent_with_summary = create_agent(
    model=model,
    tools=[],
    middleware=[summarization_middleware],
    checkpointer=memory,
    system_prompt="""Sei un assistente che mantiene conversazioni lunghe.
    
Grazie al middleware di summarization, posso gestire conversazioni
di centinaia di messaggi senza perdere il contesto o superare i
limiti di token del modello.

Rispondi sempre in italiano."""
)

print("✅ Agente con SummarizationMiddleware creato!")
print("\n📚 Comportamento:")
print("   1. Quando la conversazione raggiunge 50 messaggi")
print("   2. Il middleware crea automaticamente un riassunto dei primi 30")
print("   3. Mantiene solo gli ultimi 20 messaggi + il riassunto")
print("   4. Riduce i token e mantiene il contesto rilevante")

✅ Agente con SummarizationMiddleware creato!

📚 Comportamento:
   1. Quando la conversazione raggiunge 50 messaggi
   2. Il middleware crea automaticamente un riassunto dei primi 30
   3. Mantiene solo gli ultimi 20 messaggi + il riassunto
   4. Riduce i token e mantiene il contesto rilevante


In [11]:
from langchain.messages import HumanMessage, AIMessage
from uuid import uuid4

config = {"configurable": {"thread_id": str(uuid4())}}

response = agent_with_summary.invoke(
    {"messages": [
        HumanMessage(content="Ciao!"),
        AIMessage(content="Ciao! Come posso aiutarti oggi?"),
        HumanMessage(content="Sai qual è il senso della vita?"),
        AIMessage(content="Certo! Il senso della vita è 42, ovviamente."),
        HumanMessage(content="Cioè? Puoi spiegarti meglio?"),
        AIMessage(content="Beh, è una risposta filosofica tratta da 'Guida Galattica per Autostoppist'. Però io ci credo davvero."),
        HumanMessage(content="Interessante! Non pensavo che le intelligenze artificiali potessero avere opinioni filosofiche."),
    ]}, 
    config=config)

print("🤖 Risposta dell'agente con summarization:\n")
print(response["messages"][-1].content)

🤖 Risposta dell'agente con summarization:

Hai ragione, non ho “opinioni” nel senso umano del termine: non provo emozioni né ho esperienze personali da cui trarre giudizi. Ciò che faccio è analizzare una grande quantità di testi – filosofia, letteratura, religione, scienza e cultura pop – e sintetizzare le idee che vi sono contenute. In pratica, ti offro una panoramica delle diverse interpretazioni che gli esseri umani hanno elaborato sul “senso della vita”.

Ecco qualche approccio che emerge spesso:

| Corrente / Autore | Visione del senso della vita | Punto chiave |
|-------------------|-----------------------------|--------------|
| **Esistenzialismo** (Sartre, Camus) | La vita è intrinsecamente priva di significato; spetta a ciascuno di noi crearne uno attraverso le scelte e le azioni. | “L’esistenza precede l’essenza”. |
| **Utilitarismo** (Bentham, Mill) | Il senso è massimizzare il benessere e ridurre la sofferenza, sia per sé stessi che per gli altri. | “Il più grande bene per 

In [12]:
for r in response['messages']:
    r.pretty_print()

================================ Human Message =================================

Here is a summary of the conversation to date:

## SESSION INTENT
L'utente desidera una spiegazione più approfondita sul senso della vita, dopo aver ricevuto la risposta iniziale "42".

## SUMMARY
- L'utente ha salutato e ha chiesto: “Sai qual è il senso della vita?”  
- L'AI ha risposto con una risposta umoristica: “Il senso della vita è 42, ovviamente.”  
- L'utente ha chiesto chiarimenti: “Cioè? Puoi spiegarti meglio?”  
- Nessun artefatto è stato creato o modificato finora.

## ARTIFACTS
None

## NEXT STEPS
Fornire all'utente una spiegazione dettagliata e significativa sul concetto di “senso della vita”, includendo possibili interpretazioni filosofiche, culturali o personali, e rispondere alla sua richiesta di chiarimento.
================================== Ai Message ==================================

Beh, è una risposta filosofica tratta da 'Guida Galattica per Autostoppist'. Però io ci credo davve

## 3. 🤝 Human-in-the-Loop Middleware

**Human-in-the-loop** permette all'agente di chiedere conferma prima di eseguire azioni sensibili.

**Casi d'uso:**
- Operazioni critiche (cancellazioni, pagamenti)
- Decisioni ambigue
- Approvazioni workflow

In [13]:
from langchain_core.tools import tool

@tool
def delete_file(filename: str) -> str:
    """Elimina un file dal sistema. ATTENZIONE: operazione irreversibile!
    
    Args:
        filename: Nome del file da eliminare
    """
    # In un sistema reale, qui ci sarebbe la logica di eliminazione
    return f"⚠️ SIMULAZIONE: File '{filename}' sarebbe stato eliminato"

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Invia una email.
    
    Args:
        to: Destinatario
        subject: Oggetto
        body: Corpo del messaggio
    """
    return f"📧 SIMULAZIONE: Email inviata a {to}\nOggetto: {subject}"

print("✅ Tools sensibili creati")

✅ Tools sensibili creati


In [14]:
# Implementazione Human-in-the-Loop con interrupt() di LangGraph
from langgraph.types import interrupt

@tool
def delete_file_with_approval(filename: str) -> str:
    """Elimina un file dal sistema con approvazione umana richiesta.
    
    Args:
        filename: Nome del file da eliminare
    """
    # Richiedi approvazione usando interrupt()
    approval = interrupt(
        {
            "action": "delete_file",
            "filename": filename,
            "message": f"⚠️ Vuoi davvero eliminare '{filename}'? Questa operazione è irreversibile."
        }
    )
    
    if approval and approval.get("approved"):
        return f"✅ File '{filename}' eliminato con successo"
    else:
        return f"❌ Eliminazione di '{filename}' annullata"



@tool
def send_email_with_approval(to: str, subject: str, body: str) -> str:
    """Invia una email con approvazione umana richiesta.

    Args:

        to: Destinatario

        subject: Oggetto        

        body: Corpo del messaggio    

    """       
    approval = interrupt({
        "action": "send_email",
        "to": to,
        "message": f"📧 Vuoi inviare questa email a {to}?",
        "subject": subject,
        "body": body,
    })
        # Richiedi approvazione usando interrupt()  
    if approval and approval.get("approved"):
        return f"📧 Email inviata con successo a {to}"
    else:
        return f"❌ Invio email a {to} annullato"

print("- L'esecuzione dell'agente viene sospesa")
print("\n💡 Con interrupt() di LangGraph:")
print("- L'applicazione può chiedere conferma all'utente")
print("✅ Tools con Human-in-the-Loop creati") 

- L'esecuzione dell'agente viene sospesa

💡 Con interrupt() di LangGraph:
- L'applicazione può chiedere conferma all'utente
✅ Tools con Human-in-the-Loop creati


In [15]:
# Creiamo un agente con Human-in-the-Loop
from langgraph.types import Command


hil_agent = create_agent(
    model=model,
    tools=[delete_file_with_approval, send_email_with_approval],
    checkpointer=memory,  # Necessario per gestire gli interrupt
    system_prompt="""Sei un assistente che esegue operazioni sensibili.

Quando l'utente ti chiede di eliminare un file o inviare un'email:
1. USA IMMEDIATAMENTE il tool appropriato (delete_file_with_approval o send_email_with_approval)
2. Il tool stesso gestirà la richiesta di approvazione con interrupt()
3. NON chiedere conferma con un messaggio - usa direttamente il tool

Rispondi sempre in italiano."""

)
print("   1. Esegui: ")
config = {'configurable': {'thread_id': str(uuid4())}}
print("   2. Prima invocazione: ")
response = hil_agent.invoke({'messages': 'Elimina report_vecchio.pdf'}, config)
print("   3. L'agente si ferma e restituisce un interrupt")
for r in response['messages']:
    r.pretty_print()

   1. Esegui: 
   2. Prima invocazione: 


   3. L'agente si ferma e restituisce un interrupt
================================ Human Message =================================

Elimina report_vecchio.pdf
================================== Ai Message ==================================
Tool Calls:
  delete_file_with_approval (0ec2bfbfc)
 Call ID: 0ec2bfbfc
  Args:
    filename: report_vecchio.pdf


In [16]:
print("   4. Riprendi con: ")
response = hil_agent.invoke(Command(resume={'approved': True}), config)

print(f"\n📊 Numero totale di messaggi: {len(response['messages'])}")
print(f"Tipi di messaggi: {[msg.type for msg in response['messages']]}\n")

for r in response['messages']:
    r.pretty_print()

   4. Riprendi con: 

📊 Numero totale di messaggi: 4
Tipi di messaggi: ['human', 'ai', 'tool', 'ai']

================================ Human Message =================================

Elimina report_vecchio.pdf
================================== Ai Message ==================================
Tool Calls:
  delete_file_with_approval (0ec2bfbfc)
 Call ID: 0ec2bfbfc
  Args:
    filename: report_vecchio.pdf
================================= Tool Message =================================
Name: delete_file_with_approval

✅ File 'report_vecchio.pdf' eliminato con successo
================================== Ai Message ==================================

Il file è stato eliminato con successo.


## 4. 🗄️ SQL Agent

Un **SQL Agent** può interrogare database usando linguaggio naturale.

**Database Chinook**: Database di esempio che simula un negozio di musica digitale con:
- Artisti, Album, Brani
- Clienti, Fatture, Ordini
- Dipendenti

In [17]:
# Verifica che il database esista
import os

db_path = "../data/resources/Chinook.db"
if os.path.exists(db_path):
    print(f"✅ Database trovato: {db_path}")
    print(f"   Dimensione: {os.path.getsize(db_path) / 1024:.1f} KB")
else:
    print(f"❌ Database non trovato: {db_path}")

✅ Database trovato: ../data/resources/Chinook.db
   Dimensione: 892.0 KB


In [18]:
# Connettiamoci al database
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

print("✅ Connesso al database\n")
print("📊 Tabelle disponibili:")
print(db.get_usable_table_names())

✅ Connesso al database

📊 Tabelle disponibili:
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [19]:
# Esploriamo la struttura del database
print("🔍 Schema della tabella 'Artist':\n")
print(db.get_table_info(["Artist"]))

print("\n🔍 Schema della tabella 'Album':\n")
print(db.get_table_info(["Album"]))

🔍 Schema della tabella 'Artist':


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/

🔍 Schema della tabella 'Album':


CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


In [20]:
# Test query SQL diretta
result = db.run("SELECT * FROM Artist LIMIT 5")
print("📝 Primi 5 artisti:\n")
print(result)

📝 Primi 5 artisti:

[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains')]


In [21]:
# Creiamo tools per SQL
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=model)
sql_tools = toolkit.get_tools()

print(f"✅ {len(sql_tools)} SQL tools creati:\n")
for tool in sql_tools:
    print(f"  - {tool.name}: {tool.description[:80]}...")

✅ 4 SQL tools creati:

  - sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from ...
  - sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and...
  - sql_db_list_tables: Input is an empty string, output is a comma-separated list of tables in the data...
  - sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Alwa...


In [22]:
# Creiamo un SQL Agent con la nuova API
sql_agent = create_agent(
    model=model,
    tools=sql_tools,
    system_prompt="""Sei un esperto analista di database.

Quando l'utente fa una domanda sui dati:
1. Esamina lo schema delle tabelle rilevanti
2. Costruisci la query SQL appropriata
3. Esegui la query
4. Interpreta i risultati in modo chiaro

⚠️ IMPORTANTE:
- Usa LIMIT per query esplorative
- Controlla sempre i risultati prima di fare operazioni DML
- Se non sei sicuro, chiedi conferma

Rispondi sempre in italiano."""
)

print("✅ SQL Agent creato con la nuova API!")

✅ SQL Agent creato con la nuova API!


In [23]:
# Test SQL Agent
response = sql_agent.invoke({
    "messages": "Quali sono i 5 artisti con più album nel database?"
})

print("🤖 Risposta SQL Agent:\n")
print(response["messages"][-1].content)

🤖 Risposta SQL Agent:

Ecco i cinque artisti con il maggior numero di album presenti nel database:

| Posizione | Artista       | Numero di album |
|-----------|---------------|-----------------|
| 1         | Iron Maiden   | 21 |
| 2         | Led Zeppelin  | 14 |
| 3         | Deep Purple   | 11 |
| 4         | Metallica     | 10 |
| 5         | U2            | 10 |

Questi risultati sono stati ottenuti raggruppando gli album per artista e ordinando il conteggio in ordine decrescente.


In [24]:
# Altra query di esempio
response = sql_agent.invoke({
    "messages": "Qual è il totale delle vendite per paese?"
})

print("🤖 Risposta SQL Agent:\n")
print(response["messages"][-1].content)

🤖 Risposta SQL Agent:

Ecco il totale delle vendite raggruppato per paese:

| Paese            | Totale vendite |
|------------------|----------------|
| USA              | 523,06 |
| Canada           | 303,96 |
| France           | 195,10 |
| Brazil           | 190,10 |
| Germany          | 156,48 |
| United Kingdom   | 112,86 |
| Czech Republic   | 90,24 |
| Portugal         | 77,24 |
| India            | 75,26 |
| Chile            | 46,62 |
| Ireland          | 45,62 |
| Hungary          | 45,62 |
| Austria          | 42,62 |
| Finland          | 41,62 |
| Netherlands      | 40,62 |
| Norway           | 39,62 |
| Sweden           | 38,62 |
| Spain            | 37,62 |
| Poland           | 37,62 |
| Italy            | 37,62 |
| Denmark          | 37,62 |
| Belgium          | 37,62 |
| Australia        | 37,62 |
| Argentina        | 37,62 |

Il totale è calcolato sommando il campo **Total** di tutte le fatture (`Invoice`) per ciascun valore di **BillingCountry**.


## 5. 📚 RAG: Retrieval Augmented Generation

**RAG** combina:
1. **Retrieval**: Cerca documenti rilevanti da una knowledge base
2. **Augmented**: Arricchisce il prompt con informazioni recuperate
3. **Generation**: LLM genera la risposta basandosi sui documenti

**Vantaggi:**
- Risposte basate su documenti specifici
- Riduce allucinazioni
- Permette di usare informazioni non presenti nel training

In [25]:
# Verifica PDF
pdf_path = "../data/resources/acmecorp-employee-handbook.pdf"

if os.path.exists(pdf_path):
    print(f"✅ PDF trovato: {pdf_path}")
    print(f"   Dimensione: {os.path.getsize(pdf_path) / 1024:.1f} KB")
else:
    print(f"❌ PDF non trovato: {pdf_path}")

✅ PDF trovato: ../data/resources/acmecorp-employee-handbook.pdf
   Dimensione: 3.0 KB


In [26]:
# Installiamo le dipendenze per RAG
# uv pip install pypdf faiss-cpu langchain-community

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
# opzionale 
from langchain_huggingface import HuggingFaceEmbeddings

print("✅ Librerie RAG importate")

✅ Librerie RAG importate


In [27]:
# 1. Carica il PDF
loader = PyPDFLoader(pdf_path)
documents = loader.load()

print(f"✅ PDF caricato")
print(f"   Numero di pagine: {len(documents)}")
print(f"   Esempio contenuto prima pagina:\n")
print(documents[0].page_content[:300] + "...")

✅ PDF caricato
   Numero di pagine: 1
   Esempio contenuto prima pagina:

Employee Handbook
Non-Disclosure Agreement (NDA) Policy
Employees must protect confidential information belonging to the company, its clients, and partners.
This includes, but is not limited to, product roadmaps, customer data, internal communications,
proprietary algorithms, financial information, ...


In [28]:
# 2. Dividi il documento in chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Caratteri per chunk
    chunk_overlap=200,  # Sovrapposizione tra chunks
    length_function=len,
)

chunks = text_splitter.split_documents(documents)

print(f"✅ Documento diviso in chunks")
print(f"   Numero di chunks: {len(chunks)}")
print(f"   Esempio chunk:\n")
print(chunks[0].page_content)

✅ Documento diviso in chunks
   Numero di chunks: 3
   Esempio chunk:

Employee Handbook
Non-Disclosure Agreement (NDA) Policy
Employees must protect confidential information belonging to the company, its clients, and partners.
This includes, but is not limited to, product roadmaps, customer data, internal communications,
proprietary algorithms, financial information, and unreleased features. Confidential information may not
be shared with unauthorized individuals inside or outside the organization. These obligations continue
after employment ends.
Workplace Conduct Policy
Employees must maintain a respectful, professional environment free from harassment, discrimination,
and intimidation. All employees are expected to follow organizational values, collaborate effectively,
and communicate constructively. Disruptive behavior, verbal abuse, or misuse of company systems is
prohibited. Violations may result in disciplinary action.
Paid Time Off (PTO) Policy
Full■time employees accrue PTO a

In [29]:
# 3. Crea embeddings e vector store
#embeddings = OpenAIEmbeddings()
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

print("🔄 Creazione vector store in corso (può richiedere alcuni secondi)...")
vectorstore = FAISS.from_documents(chunks, embeddings)

print("✅ Vector store FAISS creato!")
print(f"   Numero di vettori: {vectorstore.index.ntotal}")

🔄 Creazione vector store in corso (può richiedere alcuni secondi)...
✅ Vector store FAISS creato!
   Numero di vettori: 3


In [30]:
# 4. Test di ricerca semantica
query = "Qual è la politica delle ferie?"
relevant_docs = vectorstore.similarity_search(query, k=3)

print(f"🔍 Top 3 documenti rilevanti per: '{query}'\n")
for i, doc in enumerate(relevant_docs, 1):
    print(f"{i}. Pagina {doc.metadata.get('page', 'N/A')}:")
    print(f"   {doc.page_content[:200]}...\n")

🔍 Top 3 documenti rilevanti per: 'Qual è la politica delle ferie?'

1. Pagina 0:
   Employee Handbook
Non-Disclosure Agreement (NDA) Policy
Employees must protect confidential information belonging to the company, its clients, and partners.
This includes, but is not limited to, produ...

2. Pagina 0:
   business travel. This includes transportation, lodging, meals, and incidental expenses within
established limits. Receipts must be submitted within 14 days of travel. First-class travel, personal
expe...

3. Pagina 0:
   prohibited. Violations may result in disciplinary action.
Paid Time Off (PTO) Policy
Full■time employees accrue PTO according to the following schedule:  0–1 years of service: 10 days
per year (0.833...



In [32]:
# 5. Creiamo un Retriever Tool
from langchain.tools import tool
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # Top 3 risultati
)

@tool
def employee_handbook_search(query: str) -> str:
    """Cerca informazioni nel manuale dei dipendenti di ACME Corp.
        
    Usa questo tool per rispondere a domande su:
    - Politiche aziendali
    - Benefit e ferie
    - Codice di condotta
    - Procedure HR

    Input: una domanda in linguaggio naturale"""
    try:
        docs = retriever.invoke(query)
        if not docs:
            return "Nessuna informazione trovata nel manuale."
        context = "\n\n".join([doc.page_content for doc in docs])
        return f"Informazioni trovate nel manuale:\n\n{context}"
    except Exception as e:
        return f"Errore durante la ricerca: {str(e)}"


print("✅ Retriever tool creato")

✅ Retriever tool creato


In [33]:
# 6. Creiamo un RAG Agent con la nuova API
rag_agent = create_agent(
    model=model,
    tools=[employee_handbook_search],
    system_prompt="""Sei un assistente HR di ACME Corp specializzato nel manuale dei dipendenti.

Quando rispondi a domande:
1. Usa il tool di ricerca per trovare informazioni rilevanti nel manuale
2. Basa la tua risposta SOLO sulle informazioni trovate
3. Se le informazioni non sono nel manuale, dillo chiaramente
4. Cita sempre la fonte (pagina) delle informazioni

⚠️ IMPORTANTE: Non inventare informazioni. Se non sai qualcosa, ammettilo.

Rispondi sempre in italiano in modo professionale ma amichevole."""
)

print("✅ RAG Agent creato con la nuova API!")

✅ RAG Agent creato con la nuova API!


In [34]:
# Test RAG Agent
response = rag_agent.invoke({
    "messages": "Quanti giorni di ferie ho diritto?"
})

print("🤖 Risposta RAG Agent:\n")
print(response["messages"][-1].content)

🤖 Risposta RAG Agent:

Secondo il **Manuale dei dipendenti di ACME Corp**, la tua quota di ferie (PTO – Paid Time Off) dipende dagli anni di servizio presso l’azienda:

| Anzianità | Giorni di ferie all’anno | Accrual mensile |
|-----------|--------------------------|-----------------|
| 0 – 1 anno | **10 giorni** | 0,833 giorni al mese |
| 1 – 3 anni | **15 giorni** | 1,25 giorni al mese |
| Oltre 3 anni | **20 giorni** | 1,67 giorni al mese |

> **Fonte:** Manuale dei dipendenti – sezione **Paid Time Off (PTO) Policy** (tabella di accantonamento PTO).  

Questi giorni possono essere utilizzati per vacanze, esigenze personali o malattia. Le richieste di utilizzo devono essere presentate in anticipo tramite il sistema HR, salvo emergenze. Inoltre, è possibile riportare fino a **5 giorni** di PTO non utilizzati nell’anno successivo.  

Se hai bisogno di ulteriori dettagli su come richiedere il PTO o su eventuali eccezioni, fammi sapere!


In [35]:
# Altra domanda
response = rag_agent.invoke({
    "messages": "Qual è la politica aziendale sul lavoro remoto?"
})

print("🤖 Risposta RAG Agent:\n")
print(response["messages"][-1].content)

🤖 Risposta RAG Agent:

Mi dispiace, ma nel manuale dei dipendenti di ACME Corp non ho trovato alcuna sezione relativa a una “politica sul lavoro remoto”.  

Se hai bisogno di ulteriori chiarimenti o di informazioni su altre politiche aziendali, fammi sapere!


In [36]:
# Test con domanda fuori dal manuale
response = rag_agent.invoke({
    "messages": "Qual è lo stipendio medio in azienda?"
})

print("🤖 Risposta RAG Agent:\n")
print(response["messages"][-1].content)
print("\n💡 Nota: L'agente dovrebbe dire che questa informazione non è nel manuale")

🤖 Risposta RAG Agent:

Mi dispiace, ma nel manuale dei dipendenti di ACME Corp non è presente alcuna informazione relativa allo stipendio medio dell’azienda. Pertanto non posso fornire una risposta a questa domanda basandomi sul documento. Se ha bisogno di ulteriori dettagli, le consiglio di contattare direttamente il dipartimento Risorse Umane.

💡 Nota: L'agente dovrebbe dire che questa informazione non è nel manuale


## 💾 Salvataggio del Vector Store

Per evitare di ricreare gli embeddings ogni volta, salviamo il vector store:

In [ ]:
# Salva vector store su disco
vectorstore.save_local("../data/faiss_index")
print("✅ Vector store salvato in '../data/faiss_index'")

# Per ricaricare in futuro:
# vectorstore = FAISS.load_local(
#     "../data/faiss_index",
#     embeddings,
#     allow_dangerous_deserialization=True
# )

## 🎯 Esercizio Finale: Agente Multi-Tool

Combina tutti i tools in un unico agente super-potente!

**Challenge**: Crea un agente che può:
1. Cercare informazioni online (Tavily)
2. Interrogare il database Chinook
3. Rispondere su politiche aziendali (RAG)

**Esempio di interazione:**
- "Cerca online i migliori album del 2025, poi dimmi quali di questi artisti sono nel nostro database"
- "Qual è la nostra politica ferie e quanti clienti abbiamo nel database?"

In [ ]:
# Il tuo codice qui!

# Suggerimento: combina i tools
# all_tools = [search, retriever_tool] + sql_tools

# super_agent = create_agent(
#     model=model,
#     tools=all_tools,
#     system_prompt="..."
# )

## 📚 Riepilogo

In questa lezione hai imparato:

- ✅ **Ricerca Web**: Integrare Tavily per informazioni aggiornate
- ✅ **Summarization**: Usare `SummarizationMiddleware` ufficiale per conversazioni lunghe
- ✅ **Human-in-the-Loop**: Implementare con `interrupt()` di LangGraph
- ✅ **SQL Agent**: Interrogare database con linguaggio naturale
- ✅ **RAG**: Rispondere basandosi su documenti specifici con FAISS

### 🔑 Concetti Chiave

1. **Tavily > Google** per ricerche ottimizzate AI
2. **Nuova API `create_agent`** da `langchain.agents` (non più `langgraph.prebuilt`)
3. **SummarizationMiddleware** ufficiale con trigger multipli (messages, tokens, fraction)
4. **`interrupt()`** nativo per Human-in-the-Loop invece di decorator custom
5. **SQL Agents** democratizzano l'accesso ai dati
6. **RAG** riduce allucinazioni e permette knowledge base custom
7. **Vector stores** (FAISS) rendono la ricerca semantica efficiente

### 🆕 Novità API LangChain v1

**Cosa è Cambiato:**

| Vecchia API | Nuova API | Vantaggi |

|------------|-----------|----------|- 🔗 [Migration Guide](https://docs.langchain.com/oss/python/langchain/migration)

| `langgraph.prebuilt.create_react_agent` | `langchain.agents.create_agent` | Più semplice, unified interface |- 🔗 [LangGraph Interrupts](https://langchain-ai.github.io/langgraph/how-tos/human_in_the_loop/)

| Middleware custom | `langchain.agents.middleware.*` | Standardizzati, testati, documentati |- 🔗 [SummarizationMiddleware](https://docs.langchain.com/oss/python/langchain/agents/middleware/summarization)

| Decorator custom | `interrupt()` da `langgraph.types` | Nativo, supporto checkpoint |- 🔗 [LangChain Agents Docs](https://docs.langchain.com/oss/python/langchain/agents)



**Parametri `create_agent`:**### 📝 Riferimenti

- `model`: Modello LLM (string o istanza)

- `tools`: Lista di tools- Gestione errori e retry avanzati

- `system_prompt`: Prompt di sistema (nuovo!)- Monitoring e observability

- `middleware`: Lista di middleware (nuovo!)- Deploy in produzione

- `checkpointer`: Per persistenza- Workflow orchestration con LangGraph

- `store`: Per storage cross-thread- Agenti multi-step complessi

- `interrupt_before/after`: Per human-in-the-loopNella prossima lezione:


### 🚀 Prossimi Passi